In [ ]:
import pandas as pd
import re
import openpyxl


In [ ]:
# --- Configuración ---
RUTA_CATALOGO_SCIAN_EXCEL = './datos/scian_catalogos/SCIAN_2023_CATALOGO.xlsx' # Reemplaza
NOMBRE_HOJA_SCIAN = 'SCIAN 2023' # Reemplaza si es necesario
# Asumimos que las columnas con Código y Título/Descripción son las primeras (A y B)
# Pandas las leerá con los nombres que tengan en la primera fila de datos válidos,
# o como 'Unnamed: 0', 'Unnamed: 1' si no hay encabezados claros en esas filas.
# Vamos a cargar y luego inspeccionar los nombres.

print(f"Intentando cargar el catálogo SCIAN desde: {RUTA_CATALOGO_SCIAN_EXCEL}, Hoja: {NOMBRE_HOJA_SCIAN}")

In [ ]:
# --- 1. Cargar el Catálogo SCIAN desde Excel ---
try:
    df_scian_raw = pd.read_excel(RUTA_CATALOGO_SCIAN_EXCEL, sheet_name=NOMBRE_HOJA_SCIAN)
    print(f"Catálogo SCIAN cargado. Total de filas iniciales: {len(df_scian_raw)}")
    print("Primeras filas del catálogo RAW:")
    print(df_scian_raw.head())
    print("\nColumnas detectadas en el Excel RAW:")
    print(df_scian_raw.columns.tolist())
    
    COLUMNA_CODIGO_SCIAN_ORIGINAL = 'Código' 
    COLUMNA_DESCRIPCION_SCIAN_ORIGINAL = 'Descripción'

    
    print(f"\nUsando '{COLUMNA_CODIGO_SCIAN_ORIGINAL}' como columna de Código.")
    print(f"Usando '{COLUMNA_DESCRIPCION_SCIAN_ORIGINAL}' como columna de Descripción detallada.")

    # Crear un DataFrame limpio solo con las columnas necesarias y renombrarlas
    # Asegurarse de que estas columnas realmente existen antes de seleccionarlas
    if COLUMNA_CODIGO_SCIAN_ORIGINAL not in df_scian_raw.columns:
        raise ValueError(f"Columna de código '{COLUMNA_CODIGO_SCIAN_ORIGINAL}' no encontrada. Columnas: {df_scian_raw.columns.tolist()}")
    if COLUMNA_DESCRIPCION_SCIAN_ORIGINAL not in df_scian_raw.columns:
        raise ValueError(f"Columna de descripción '{COLUMNA_DESCRIPCION_SCIAN_ORIGINAL}' no encontrada. Columnas: {df_scian_raw.columns.tolist()}")
        
    df_scian = df_scian_raw[[COLUMNA_CODIGO_SCIAN_ORIGINAL, COLUMNA_DESCRIPCION_SCIAN_ORIGINAL]].copy()
    df_scian.rename(columns={
        COLUMNA_CODIGO_SCIAN_ORIGINAL: 'CODIGO_SCIAN_RAW',
        COLUMNA_DESCRIPCION_SCIAN_ORIGINAL: 'DESCRIPCION_RAW'
    }, inplace=True)

except FileNotFoundError:
    print(f"Error: No se encontró el archivo Excel del catálogo SCIAN en '{RUTA_CATALOGO_SCIAN_EXCEL}'")
    raise
except Exception as e:
    print(f"Error al cargar el catálogo SCIAN desde Excel: {e}")
    raise

In [ ]:
# --- 2. Limpieza y Preparación de Datos SCIAN ---
print("\n--- Limpieza y Preparación de Datos SCIAN ---")

# Eliminar filas donde el código Y la descripción sean completamente NaN
# (esto debería eliminar las filas completamente vacías)
df_scian.dropna(subset=['CODIGO_SCIAN_RAW', 'DESCRIPCION_RAW'], how='all', inplace=True)
print(f"Filas después de eliminar NaNs en código Y descripción: {len(df_scian)}")

# También es útil eliminar filas donde el CÓDIGO sea NaN, ya que esas no nos sirven.
df_scian.dropna(subset=['CODIGO_SCIAN_RAW'], inplace=True)
print(f"Filas después de eliminar NaNs en CÓDIGO: {len(df_scian)}")


# Convertir códigos a string y limpiar
df_scian['CODIGO_SCIAN_STR'] = df_scian['CODIGO_SCIAN_RAW'].astype(str).str.strip().str.replace(r'\.0$', '', regex=True)

# Convertir descripciones a string y limpiar (quitar espacios extra)
# Rellenar NaNs en DESCRIPCION_RAW con un string vacío antes de convertir a string y limpiar, 
# para evitar errores si alguna descripción es genuinamente NaN pero el código es válido.
df_scian['DESCRIPCION_LIMPIA'] = df_scian['DESCRIPCION_RAW'].fillna('').astype(str).str.strip()


# Filtrar para quedarnos solo con códigos que parezcan ser de 6 dígitos (clases de actividad)
df_scian_6_digitos = df_scian[df_scian['CODIGO_SCIAN_STR'].str.match(r'^\d{6}$')].copy()
print(f"Filas después de filtrar por códigos de 6 dígitos: {len(df_scian_6_digitos)}")

if df_scian_6_digitos.empty:
    print("ADVERTENCIA: No se encontraron códigos SCIAN de 6 dígitos. Revisa el filtrado y la columna de códigos.")
    print("Mostrando algunos valores de 'CODIGO_SCIAN_STR' para depurar (antes del filtro de 6 dígitos):")
    print(df_scian['CODIGO_SCIAN_STR'].value_counts().head(20))
    # Es posible que los códigos ya estén limpios o que la regex necesite ajuste
    # si el formato de los códigos es diferente (ej. no son puramente numéricos)
else:
    print("Primeras filas del catálogo SCIAN filtrado (6 dígitos):")
    print(df_scian_6_digitos[['CODIGO_SCIAN_STR', 'DESCRIPCION_LIMPIA']].head())


In [ ]:
# --- 3. Definición de Categorías PDI y Palabras Clave para Búsqueda ---
# Aquí defines tus categorías de interés y una lista de palabras clave (o frases)
# que ayudarán a identificar actividades relevantes para cada una.
# Las palabras clave deben estar en minúsculas para la búsqueda insensible a mayúsculas.
# Usa varios sinónimos o términos relacionados.

categorias_palabras_clave = {
    'competidores_directos_conv': ['tiendas de conveniencia', 'minisúper', 'supercito', 'autoservicio'], # OXXO, 7-Eleven, etc.
    'competidores_indirectos_abarrotes': ['abarrotes', 'ultramarinos', 'miscelánea con venta de'],
    'competidores_indirectos_farmacias_conv': ['farmacias con minisúper', 'farmacia y conveniencia'],
    
    'generador_trafico_escuelas': ['escuela', 'colegio', 'instituto', 'enseñanza', 'preescolar', 'primaria', 'secundaria'],
    'generador_trafico_universidades': ['universidad', 'educación superior', 'tecnológico superior'],
    'generador_trafico_oficinas': ['oficinas corporativas', 'servicios profesionales', 'consultoría', 'despacho', 'servicios de administración de negocios'],
    'generador_trafico_hospitales_clinicas': ['hospital', 'clínica', 'consultorios médicos', 'servicios de salud'],
    'generador_trafico_bancos': ['banca múltiple', 'servicios bancarios', 'cajas de ahorro'], # ATM es más OSM
    'generador_trafico_restaurantes_fastfood': ['restaurantes de comida rápida', 'pizzas para llevar', 'hamburguesas para llevar', 'pollos rostizados para llevar', 'alimentos para llevar', 'cafetería'],
    'generador_trafico_transporte': ['terminal de autobuses', 'estación de autobuses', 'transporte de pasajeros'], # Paradas es más OSM
    'generador_trafico_hoteles': ['hotel', 'motel', 'servicios de alojamiento'],
    'generador_trafico_gasolineras': ['gasolineras', 'estaciones de servicio', 'comercio al por menor de gasolina']
}

In [ ]:
# --- 4. Búsqueda y Agrupación Automática de Códigos SCIAN ---
print("\n--- Búsqueda y Agrupación Automática de Códigos SCIAN ---")
categorias_pdi_auto = {}

if not df_scian_6_digitos.empty:
    for categoria, keywords in categorias_palabras_clave.items():
        print(f"\nBuscando para categoría: {categoria} (Palabras clave: {', '.join(keywords)})")
        
        # Construir una expresión regular para buscar CUALQUIERA de las palabras clave
        # \b asegura que sean palabras completas (o al inicio/final) para evitar matches parciales no deseados
        regex_pattern = '|'.join([r'\b' + re.escape(kw) + r'\b' for kw in keywords])
    
        # Filtrar descripciones que contengan alguna de las palabras clave
        df_matches = df_scian_6_digitos[
            df_scian_6_digitos['DESCRIPCION_LIMPIA'].str.contains(regex_pattern, case=False, na=False, regex=True)
        ]
        
        codigos_encontrados = df_matches['CODIGO_SCIAN_STR'].unique().tolist()
        categorias_pdi_auto[categoria] = codigos_encontrados
        
        if codigos_encontrados:
            print(f"  Códigos encontrados ({len(codigos_encontrados)}): {', '.join(codigos_encontrados[:20])}" + ("..." if len(codigos_encontrados) > 20 else ""))
            # print("  Ejemplos de descripciones encontradas:")
            # print(df_matches[['CODIGO_SCIAN_STR', 'DESCRIPCION_LIMPIA']].head(3).to_string())
        else:
            print("  No se encontraron códigos para esta categoría con las palabras clave actuales.")
else:
    print("DataFrame 'df_scian_6_digitos' está vacío. No se puede realizar la búsqueda automática.")

In [ ]:

# --- 5. Revisión y Refinamiento del Diccionario `categorias_pdi_auto` ---
print("\n--- Diccionario `categorias_pdi_auto` Generado (NECESITA REVISIÓN MANUAL) ---")

for categoria, codigos in categorias_pdi_auto.items():
    print("\n----------------------------------------------------------------------")
    print(f"Categoría Propuesta: {categoria.upper()}")
    print("----------------------------------------------------------------------")
    if not codigos:
        print("  (Sin códigos encontrados para esta categoría con las palabras clave actuales)")
        continue

    descripciones_df = df_scian_6_digitos[
        df_scian_6_digitos['CODIGO_SCIAN_STR'].isin(codigos)
    ][['CODIGO_SCIAN_STR', 'DESCRIPCION_LIMPIA']].copy()

    if not descripciones_df.empty:
        for index, row in descripciones_df.iterrows():
            # Ajusta el padding para alinear. Elige un ancho máximo para el código.
            codigo_padding = 8 
            print(f"  Código: {row['CODIGO_SCIAN_STR']:<{codigo_padding}} Descripción: {row['DESCRIPCION_LIMPIA']}")
    else:
        print(f"  Códigos: {codigos} (No se encontraron descripciones correspondientes, ¡revisar!)")
print("----------------------------------------------------------------------")

In [ ]:
categorias_pdi_final = {
    'competidores_directos_conv': [
        '462112', # Minisupers
    ],
    'competidores_indirectos_abarrotes': [
        '461110', # Tiendas de abarrotes, ultramarinos y misceláneas (Verificar si este es el SCIAN 2023)
    ],
    'competidores_indirectos_farmacias_conv': [
        '464112'  # Farmacias con minisúper (Verificar SCIAN 2023)
    ],
    'generador_trafico_escuelas_basicas_medias': [ # Preescolar, Primaria, Secundaria, Media/Bachillerato
        '611111', '611112', 
        '611121', '611122', 
        '611131', '611132', 
        '611141', '611142', 
        '611161', '611162',
        '611171', '611172' # Escuelas con múltiples niveles (podrían incluirse)
    ],
    'generador_trafico_universidades_tecnologicos': [
        '611211', '611212', # Formación de técnicos superiores, universidades tecnológicas
        '611311', '611312'  # Universidades e institutos tecnológicos (Educación Superior)
    ],
    'generador_trafico_oficinas_corporativos': [
        '551111', # Corporativos 
        '541610', # Consultoría en administración
        '541690', # Otros servicios de consultoría
        '541990'  # Otros servicios profesionales, científicos y técnicos
 ],
    'generador_trafico_hospitales_clinicas': [
        '621115', '621116', # Clínicas de consultorios médicos (privado/público)
        '622111', '622112'  # Hospitales generales (privado/público)
    ],
    'generador_trafico_bancos': [
        '522110'  # Banca múltiple (VERIFICAR CÓDIGO SCIAN 2023)
    ],
    'generador_trafico_restaurantes_fastfood': [
        '722511',
        '722512',
        '722513', 
        '722514',
        '722515', 
        '722516',
        '722517',
        '722518',
        '722519',
    ],
    'generador_trafico_transporte_terminales': [
        '485210', # Transporte foráneo de pasajeros por autobús (Terminales) 
        '485410'  # Transporte de escolares y de personal (Relevante por el movimiento, no por la terminal)
        # Buscar códigos para estaciones de metro/tren si son relevantes para tu zona y no los sacarás de OSM
    ],
    'generador_trafico_hoteles': [
        '721111', '721112', '721113', '721120', '721190', '721210', '721311', '721312' # Parecen correctos
    ],
    'generador_trafico_gasolineras': [
        '468411' # Comercio al por menor de gasolina y diesel (Gasolineras)
    ]
}

# Guardar este diccionario final para usarlo en otros notebooks
import json
try:
    # Asegúrate de que el directorio de salida exista
    import os
    output_dir = './datos/output/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(output_dir + 'categorias_pdi_scian_final.json', 'w', encoding='utf-8') as f:
        json.dump(categorias_pdi_final, f, indent=4, ensure_ascii=False)
    print("\nDiccionario `categorias_pdi_final` guardado en './datos/output/categorias_pdi_scian_final.json'")
    print("¡Ya puedes usar este JSON en el notebook 05_Integracion_DENUE.ipynb!")
except NameError:
    print("Asegúrate de definir `categorias_pdi_final` en esta celda antes de ejecutar esto.")
except Exception as e:
    print(f"Error al guardar el diccionario final como JSON: {e}")